# Recruitment Experiment

In developing lifelong learning algorithms, prior work has involved two main approaches: building and reallocating. Building involves adding new resources to support the arrival of new data, whereas reallocation involves compression of representations to make room for new ones. However, biologically, there is a spectrum between these two modes.

In order to examine whether current resources could be better leveraged, we test a range of approaches: **recruitment** of the best-performing existing trees, **building** new trees completely (the default approach that our L2F uses), ignoring all prior trees (essentially an uncertainty forest), and a **hybrid** between building and recruitment.

We examine the performance of these four approaches based on the available training sample size.

In [24]:
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from itertools import product
import pandas as pd

import numpy as np
import pickle
from math import log2, ceil 

from joblib import Parallel, delayed

import tensorflow as tf

#import warnings
#warnings.filterwarnings(action='once')


from proglearn.progressive_learner import ProgressiveLearner
from proglearn.forest import LifelongClassificationForest, UncertaintyForest

### CIFAR 10x10 Tasks

The classification problem that we examine in this tutorial makes use of the CIFAR 10x10 dataset. This dataset contains

In [13]:
# import data 
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
data_x = np.concatenate([X_train, X_test])
data_x = data_x.reshape((data_x.shape[0], data_x.shape[1] * data_x.shape[2] * data_x.shape[3]))
data_y = np.concatenate([y_train, y_test])
data_y = data_y[:, 0]

train_x_across_task, train_y_across_task, test_x_across_task, test_y_across_task = sort_data(
    data_x,data_y,num_points_per_task
    )

169009152/169001437 [==============================] - 49s 0us/step


### The Experiment

In [10]:
def sort_data(data_x, data_y, num_points_per_task, total_task=10, shift=1):
    x = data_x.copy()
    y = data_y.copy()
    idx = [np.where(data_y == u)[0] for u in np.unique(data_y)]
    train_x_across_task = []
    train_y_across_task = []
    test_x_across_task = []
    test_y_across_task = []

    batch_per_task=5000//num_points_per_task
    sample_per_class = num_points_per_task//total_task
    test_data_slot=100//batch_per_task

    for task in range(total_task):
        for batch in range(batch_per_task):
            for class_no in range(task*10,(task+1)*10,1):
                indx = np.roll(idx[class_no],(shift-1)*100)
                
                if batch==0 and class_no==task*10:
                    train_x = x[indx[batch*sample_per_class:(batch+1)*sample_per_class],:]
                    train_y = y[indx[batch*sample_per_class:(batch+1)*sample_per_class]]
                    test_x = x[indx[batch*test_data_slot+500:(batch+1)*test_data_slot+500],:]
                    test_y = y[indx[batch*test_data_slot+500:(batch+1)*test_data_slot+500]]
                else:
                    train_x = np.concatenate((train_x, x[indx[batch*sample_per_class:(batch+1)*sample_per_class],:]), axis=0)
                    train_y = np.concatenate((train_y, y[indx[batch*sample_per_class:(batch+1)*sample_per_class]]), axis=0)
                    test_x = np.concatenate((test_x, x[indx[batch*test_data_slot+500:(batch+1)*test_data_slot+500],:]), axis=0)
                    test_y = np.concatenate((test_y, y[indx[batch*test_data_slot+500:(batch+1)*test_data_slot+500]]), axis=0)
        
        train_x_across_task.append(train_x)
        train_y_across_task.append(train_y)
        test_x_across_task.append(test_x)
        test_y_across_task.append(test_y)

    return train_x_across_task, train_y_across_task, test_x_across_task, test_y_across_task

In [11]:
def voter_predict_proba(voter, nodes_across_trees):
            def worker(tree_idx):
                #get the node_ids_to_posterior_map for this tree
                node_ids_to_posterior_map = voter.tree_idx_to_node_ids_to_posterior_map[tree_idx]

                #get the nodes of X
                nodes = nodes_across_trees[tree_idx]

                posteriors = []
                node_ids = node_ids_to_posterior_map.keys()

                #loop over nodes of X
                for node in nodes:
                    #if we've seen this node before, simply get the posterior
                    if node in node_ids:
                        posteriors.append(node_ids_to_posterior_map[node])
                    #if we haven't seen this node before, simply use the uniform posterior 
                    else:
                        posteriors.append(np.ones((len(np.unique(voter.classes_)))) / len(voter.classes_))
                return posteriors

            if voter.parallel:
                return Parallel(n_jobs=-1)(
                                delayed(worker)(tree_idx) for tree_idx in range(voter.n_estimators)
                        )

            else:
                return [worker(tree_idx) for tree_idx in range(voter.n_estimators)]

In [12]:
def estimate_posteriors(l2f, X, representation = 0, decider = 0):
        l2f.check_task_idx_(decider)
        
        if representation == "all":
            representation = range(l2f.n_tasks)
        elif isinstance(representation, int):
            representation = np.array([representation])
        
        def worker(transformer_task_idx):
            transformer = l2f.transformers_across_tasks[transformer_task_idx]
            voter = l2f.voters_across_tasks_matrix[decider][transformer_task_idx]

            return voter_predict_proba(voter,transformer(X))
        
        '''if l2f.parallel:
            posteriors_across_tasks = np.array(
                        Parallel(n_jobs=-1)(
                                delayed(worker)(transformer_task_idx) for transformer_task_idx in representation
                        )
                )    
        else:'''
        posteriors_across_tasks = np.array([worker(transformer_task_idx) for transformer_task_idx in representation])    

        return posteriors_across_tasks

In [17]:
############################
### Main hyperparameters ###
############################
ntrees = 50
hybrid_comp_trees = 25
estimation_set = 0.63
validation_set= 1-estimation_set

#num_points_per_task = 5000
#num_points_per_forest = 500
#reps = 30
num_points_per_task = 100
num_points_per_forest = 10
reps = 5

task_10_sample = 10*np.array([10, 50, 100, 200, 350, 500])

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# create matrices for storing values
hybrid = np.zeros(reps,dtype=float)
building = np.zeros(reps,dtype=float)
recruiting= np.zeros(reps,dtype=float)
uf = np.zeros(reps,dtype=float)
mean_accuracy_dict = {'hybrid':[],'building':[],'recruiting':[],'UF':[]}
std_accuracy_dict = {'hybrid':[],'building':[],'recruiting':[],'UF':[]}

#### from paper:

train L2F on first nine CIFAR 10x10 tasks (50 trees/task, 500 samples/task)

for 10th task:
1. recruiting = select 50/450 existing trees that perform best on task 10
2. building = train 50 new trees (L2F default)
3. hybrid = build and recruit 25 trees
4. UF = ignore prior trees

should see L2F outperform others except @ 5k training samples: "relative performance depends on available resources and sample size"

future work: "investigate optimal strtegies or determining how to optimally leverage existing resources given a new task"

In [51]:
#from proglearn.progressive_learner import ClassificationProgressiveLearner
from proglearn.transformers import TreeClassificationTransformer
from proglearn.voters import TreeClassificationVoter
from proglearn.deciders import SimpleArgmaxAverage


/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
import functions.recruitment_functions as fn

In [43]:
# for ns in task_10_sample: # 100 to 5000 sample size for task 10
    
#     # size of estimation and validation sample sets
#     estimation_sample_no = ceil(estimation_set*ns)
#     validation_sample_no = ns - estimation_sample_no
#     #
#     print(estimation_sample_no)
#     print(validation_sample_no)
#     #

#     # repeat `rep` times
#     for rep in range(reps):
#         print("doing {} samples for {} th rep".format(ns,rep))
        
#         ## estimation
#         l2f = LifelongClassificationForest(n_estimators=ntrees)
        
#         # training l2f on first 9 tasks
#         for task in range(9):
#             indx = np.random.choice(num_points_per_task, num_points_per_forest, replace=False)
#             l2f.add_task(
#                 train_x_across_task[task][indx], 
#                 train_y_across_task[task][indx]) 
#                 #max_depth=ceil(log2(num_points_per_forest)))
        
#         # 10th task...
        
#         task_10_train_indx = np.random.choice(num_points_per_task, ns, replace=False)

#         l2f.add_task(
#             train_x_across_task[9][task_10_train_indx[:estimation_sample_no]], 
#             train_y_across_task[9][task_10_train_indx[:estimation_sample_no]]
#             #max_depth=ceil(log2(estimation_sample_no)),
#             )

        
#         ## L2F validation
#         for task_num in range(9):
#             posterior_per_tree = l2f.predict_proba(
#                 train_x_across_task[9][task_10_train_indx[estimation_sample_no:]],
#                 task_id=task_num
#                 )
#             print(posterior_per_tree.shape)
        
#         #posteriors_across_trees = estimate_posteriors(
#         #    l2f,
#         #    train_x_across_task[9][task_10_train_indx[estimation_sample_no:]],
#         #    representation=[0,1,2,3,4,5,6,7,8],
#         #    decider=9
#         #    )
        
#         posteriors_across_trees = posteriors_across_trees.reshape(
#             9*ntrees,
#             validation_sample_no,
#             10
#             )



In [70]:
for ns in task_10_sample: # 100 to 5000 sample size for task 10
    
    # size of estimation and validation sample sets
    estimation_sample_no = ceil(estimation_set*ns)
    validation_sample_no = ns - estimation_sample_no
    #
    print(estimation_sample_no)
    print(validation_sample_no)
    #

    # repeat `rep` times
    for rep in range(reps):
        print("doing {} samples for {} th rep".format(ns,rep))
        
        ## estimation
        
        # use lower-level ProgressiveLearner instance
        l2f = ProgressiveLearner(
            default_transformer_class=TreeClassificationTransformer,
            default_transformer_kwargs={},
            default_voter_class=TreeClassificationVoter,
            default_voter_kwargs={
                "finite_sample_correction": False
            },
            #default_decider_class=SimpleArgmaxAverage,
            default_decider_class=fn.NOTAVERAGE,
            default_decider_kwargs={},
        )
        
        
        # training l2f on first 9 tasks
        for task in range(9):
            indx = np.random.choice(num_points_per_task, num_points_per_forest, replace=False)
            cur_X = train_x_across_task[task][indx]
            cur_y = train_y_across_task[task][indx]
            l2f.add_task(
                cur_X, 
                cur_y,
                num_transformers = ntrees,
                #max_depth=ceil(log2(num_points_per_forest)))
                #voter_kwargs={"classes": np.unique(cur_y),"finite_sample_correction": False},
                decider_kwargs={"classes": np.unique(cur_y)}
            )
        
        # 10th task...
        
        task_10_train_indx = np.random.choice(num_points_per_task, ns, replace=False)
        cur_X = train_x_across_task[9][task_10_train_indx[:estimation_sample_no]]
        cur_y = train_y_across_task[9][task_10_train_indx[:estimation_sample_no]]
        l2f.add_task(
            cur_X, 
            cur_y,
            num_transformers = ntrees,
            #max_depth=ceil(log2(estimation_sample_no)),
            #voter_kwargs={"classes": np.unique(cur_y),"finite_sample_correction": False},
            decider_kwargs={"classes": np.unique(cur_y)}
        )
        
        ## L2F validation
        for tasks in range(9):
            posterior_per_tree = l2f.predict_proba(
                train_x_across_task[9][task_10_train_indx[estimation_sample_no:]],
                task_id=tasks,
                transformer_ids=[0,1,2,3,4,5,6,7,8]
                )
            print(posterior_per_tree.shape)
        
        #posteriors_across_trees = estimate_posteriors(
        #    l2f,
        #    train_x_across_task[9][task_10_train_indx[estimation_sample_no:]],
        #    representation=[0,1,2,3,4,5,6,7,8],
        #    decider=9
        #    )
        
        posteriors_across_trees = posteriors_across_trees.reshape(
            9*ntrees,
            validation_sample_no,
            10
            )



63
37
doing 100 samples for 0 th rep
False


AttributeError: 'TreeClassificationVoter' object has no attribute 'predict'

In [ ]:
        error_across_trees = np.zeros(9*ntrees)
        validation_target = train_y_across_task[9][task_10_train_indx[estimation_sample_no:]]
        for tree in range(9*ntrees):
            res = np.argmax(posteriors_across_trees[tree],axis=1) + 90
            error_across_trees[tree] = 1-np.mean(
                validation_target==res
            )

        best_50_tree = np.argsort(error_across_trees)[:50]
        best_25_tree = best_50_tree[:25]
        
        ## uf trees validation
        posteriors_across_trees = estimate_posteriors(
            l2f,
            train_x_across_task[9][task_10_train_indx[estimation_sample_no:]],
            representation=9,
            decider=9
            )[0]

        error_across_trees = np.zeros(ntrees)
        validation_target = train_y_across_task[9][task_10_train_indx[estimation_sample_no:]]
        for tree in range(ntrees):
            res = np.argmax(posteriors_across_trees[tree],axis=1) + 90
            error_across_trees[tree] = 1-np.mean(
                validation_target==res
            )
        best_25_uf_tree = np.argsort(error_across_trees)[:25]

        ## evaluation
        posteriors_across_trees = estimate_posteriors(
            l2f,
            test_x_across_task[9],
            representation=[0,1,2,3,4,5,6,7,8],
            decider=9
            )
        posteriors_across_trees = posteriors_across_trees.reshape(
            9*ntrees,
            1000,
            10
            )
        # RECRUITING
        recruiting_posterior = np.mean(posteriors_across_trees[best_50_tree],axis=0)
        res = np.argmax(recruiting_posterior,axis=1) + 90
        recruiting[rep] = 1 - np.mean(
                test_y_across_task[9]==res
            )
        # BUILDING
        building_res = l2f.predict(
            test_x_across_task[9],
            representation=[0,1,2,3,4,5,6,7,8,9],
            decider=9
        )
        building[rep] = 1 - np.mean(
                test_y_across_task[9]==building_res
            )
        # UF
        uf_res = l2f.predict(
            test_x_across_task[9],
            representation=9,
            decider=9
        )
        uf[rep] = 1 - np.mean(
                test_y_across_task[9]==uf_res
            )
        # HYBRID
        posteriors_across_trees_hybrid_uf = estimate_posteriors(
            l2f,
            test_x_across_task[9],
            representation=9,
            decider=9
            )[0]
        
        hybrid_posterior_all = np.concatenate(
            (
                posteriors_across_trees[best_25_tree],
                posteriors_across_trees_hybrid_uf[best_25_uf_tree]
            ),
            axis=0
        )
        hybrid_posterior = np.mean(
            hybrid_posterior_all,
            axis=0
        )
        hybrid_res = np.argmax(hybrid_posterior,axis=1) + 90
        hybrid[rep] = 1 - np.mean(
                test_y_across_task[9]==hybrid_res
            )
    mean_accuracy_dict['hybrid'].append(np.mean(hybrid))
    std_accuracy_dict['hybrid'].append(np.std(hybrid,ddof=1))

    mean_accuracy_dict['building'].append(np.mean(building))
    std_accuracy_dict['building'].append(np.std(building,ddof=1))

    mean_accuracy_dict['recruiting'].append(np.mean(recruiting))
    std_accuracy_dict['recruiting'].append(np.std(recruiting,ddof=1))

    mean_accuracy_dict['UF'].append(np.mean(uf))
    std_accuracy_dict['UF'].append(np.std(uf,ddof=1))

summary = (mean_accuracy_dict,std_accuracy_dict)

with open('result/recruitment_exp_'+str(num_points_per_forest)+'.pickle','wb') as f:
    pickle.dump(summary,f)
# %%

### Visualizing the Results

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,8))
mean_error = unpickle('recruitment_result/recruitment_mean.pickle')
std_error = unpickle('recruitment_result/recruitment_std.pickle')
ns = 10*np.array([50, 100, 200, 350, 500])
colors = sns.color_palette('Set1', n_colors=mean_error.shape[0]+2)

#labels = ['recruiting', 'Uncertainty Forest', 'hybrid', '50 Random', 'BF', 'building']
labels = ['hybrid', 'building', 'recruiting','50 Random', 'BF', 'Uncertainty Forest' ]
not_included = ['BF', '50 Random']
    
adjust = 0
for i, error_ in enumerate(mean_error[:-1]):
    if labels[i] in not_included:
        adjust +=1
        continue
    ax.plot(ns, mean_error[i], c=colors[i+1-adjust], label=labels[i])
    ax.fill_between(ns, 
            mean_error[i] + 1.96*std_error[i], 
            mean_error[i] - 1.96*std_error[i], 
            where=mean_error[i] + 1.96*std_error[i] >= mean_error[i] - 1.96*std_error[i], 
            facecolor=colors[i+1-adjust], 
            alpha=0.15,
            interpolate=False)

ax.plot(ns, mean_error[-1], c=colors[0], label=labels[-1])
ax.fill_between(ns, 
        mean_error[-1] + 1.96*std_error[-1], 
        mean_error[-1] - 1.96*std_error[-1], 
        where=mean_error[-1] + 1.96*std_error[i] >= mean_error[-1] - 1.96*std_error[-1], 
        facecolor=colors[0], 
        alpha=0.15,
        interpolate=False)


#ax.set_title('CIFAR Recruitment Experiment', fontsize=30)
ax.set_ylabel('Accuracy', fontsize=28)
ax.set_xlabel('Number of Task 10 Samples', fontsize=30)
ax.tick_params(labelsize=28)
ax.set_ylim(0.325, 0.575)
ax.set_title("CIFAR Recruitment",fontsize=30)
ax.set_xticks([500, 2000, 5000])
ax.set_yticks([0.35, 0.45, 0.55])

ax.legend(fontsize=12)

right_side = ax.spines["right"]
right_side.set_visible(False)
top_side = ax.spines["top"]
top_side.set_visible(False)

plt.savefig('figs/recruit.pdf', dpi=500)